# Omni Chords - for Cahos

In [23]:
from collections import Counter
from itertools import accumulate
from math import log2

## Generate Omni-Chords

### Entropy calculations

Using Shannon entropy. Also using "sub-sequence entropy" and "sequence entropy":
For "sub-sequence entropy" we take length n sub sequences of a list and treat them as symbols,
and calculate the entropy of that. Sequence entropy is just the sum of sub-sequence entropies of all lengths

```python
sequence = [2, 1, 2, 2, 1]
entropy = calculate_entropy(sequence)
# 2-length sub-sequence entropy:
subseq_entropy = calculate_entropy([(2, 1), (1, 2), (2, 2), (2, 1)])
```

In [24]:
def calculate_entropy(sequence):
    # Count the frequency of each element in the sequence
    frequency = Counter(sequence)
    total_count = len(sequence)

    # Calculate the entropy
    entropy = 0
    for count in frequency.values():
        probability = count / total_count
        entropy -= probability * log2(probability)

    return entropy

def calculate_subseq_entropy(sequence, subsequence_length):
    sub_seqs = []
    for idx in range(len(sequence)-subsequence_length+1):
        sub_seqs.append(tuple(sequence[idx:idx+subsequence_length]))
    return calculate_entropy(sub_seqs)

def calculate_sequence_entropy(sequence):
    return sum(calculate_subseq_entropy(sequence, l) for l in range(len(sequence)+1))

The Scale (more aptly chord) has a few properties of interest:

- `sequence_entropy`
- `span`: distance between the lowest and the highest note in chord
- `inervals`: all interval types contained in the chord

In [25]:
class Scale:
    def __init__(self, deltas):
        self.deltas = deltas

    def __repr__(self):
        return f"{self.deltas}"

    def __str__(self):
        return f"entropy: {self.sequence_entropy:.2f}, span: {self.span}, deltas: {self.deltas}"

    @property
    def pitch_classes(self):
        return [a % 12 for a in accumulate(self.deltas, initial=0)]

    def transposed(self, start=0):
        return [a % 12 for a in accumulate(self.deltas, initial=start)]

    @property
    def span(self):
        return sum(self.deltas)

    @property
    def intervals(self):
        return list(set(self.deltas))

    @property
    def n_intervals(self):
        return len(set(self.deltas))

    @property
    def entropy(self):
        return calculate_entropy(self.deltas)

    @property
    def transition_entropy(self):
        return calculate_subseq_entropy(self.deltas, 2)

    @property
    def sequence_entropy(self):
        return calculate_sequence_entropy(self.deltas)

### Generating omni-chords subject to some constraints

There are *a lot* of possible omni-chords. So, some constraints are applied during generation
to enable early termination of the search.

- `allowed_intervals`: List of intervals that an omni-chord can contain
- `disallowed_subsequences`: Mostly to exclude chromatic clusters. No consequtive minor deconds, etc.
- `disallowed_beginnings`: Usually don't want narrow intervals between the bass and the next note.
- `max_span`: Maximum distance between the lowest and higihest note for an interval.

In [26]:
def contains_subsequence(main_list, sub_list):
    for idx in range(len(main_list) - len(sub_list) + 1):
        if main_list[idx: idx + len(sub_list)] == sub_list:
            return True
    return False

def get_scales(allowed_intervals, disallowed_subsequences=[], disallowed_beginnings=[], max_span=88):
    result = []
    get_scales_recursive(allowed_intervals, disallowed_subsequences, disallowed_beginnings, max_span, result)

    return result

def get_scales_recursive(allowed_intervals, disallowed_subsequences, disallowed_beginnings, max_span, deltas_accumulator,
                         pitch_classes=[0], deltas=[], idx=0):
    if idx == 11:
        deltas_accumulator.append(deltas)
        # normal exit
        return
    for next_interval in allowed_intervals:
        # early exits
        if (idx == 0) and (next_interval in disallowed_beginnings):
            continue
        if sum(deltas + [next_interval]) > max_span:
            continue
        if any(contains_subsequence(deltas + [next_interval], dis_seq) for dis_seq in disallowed_subsequences):
            continue
        next_note = (pitch_classes[-1] + next_interval) % 12
        if next_note not in pitch_classes:
            get_scales_recursive(allowed_intervals, disallowed_subsequences, disallowed_beginnings, max_span, deltas_accumulator,
                      pitch_classes + [next_note], deltas + [next_interval], idx + 1)

In [30]:
dis_subseq = [
    [1, 2], [2, 1], [3, 1],
    [3, 4], [4, 3],
    [6, 1], [1, 6], [5, 1], [1, 5],
    [1, 1], [2, 2], [3, 3], [4, 4],
    [5, 5, 5], [6, 6, 6], [7, 7, 7]
]
scales = [Scale(s) for s in get_scales(
    allowed_intervals=[1, 2, 3, 4, 5, 6, 7],
    disallowed_subsequences=dis_subseq,
    disallowed_beginnings=[1, 2],
    max_span=60
)]

print(f"number of scales satisfying given constraints: {len(scales)}")

number of scales satisfying given constraints: 16068


### Further filtering and sorting

Can filter and/or sort by exact number of intervals, entropy, sequence entropy, span...

In [31]:
n_intervals = 3
selection = sorted([s for s in scales if s.n_intervals == n_intervals], key=lambda x: (x.sequence_entropy, x.span), reverse=True)
show_n = 5
print(f"first {min(show_n, len(selection))} of {len(selection)} scales remaining after filtering")
for s in selection[:show_n]:
    print(s)
print("...")

first 5 of 114 scales remaining after filtering
entropy: 22.76, span: 56, deltas: [4, 5, 6, 4, 6, 5, 5, 6, 5, 4, 6]
entropy: 22.76, span: 56, deltas: [6, 4, 5, 6, 5, 5, 6, 4, 6, 5, 4]
entropy: 22.76, span: 53, deltas: [6, 3, 5, 5, 6, 3, 6, 5, 5, 3, 6]
entropy: 22.76, span: 50, deltas: [5, 5, 6, 2, 5, 2, 6, 2, 6, 5, 6]
entropy: 22.76, span: 50, deltas: [6, 5, 6, 2, 6, 2, 5, 2, 6, 5, 5]
...


### Voice leading opportunities

- Number of shared notes between two chords.
- Number of simple swaps among moving notes (ex. 1 -> 0 and 0 -> 1)
- Bass motion

In [32]:
def n_swaps(xs, ys):
    none_matches = [(a, b) for a, b in zip (xs, ys) if a != b]
    swaps = []
    for item in none_matches:
        if tuple(item[::-1]) in none_matches:
            swaps.append(item)
    
    return len(swaps)

def mark_changes(first, second):
    return [int(a != b) for a, b in zip(first, second)]

### Notes

Create a `VoiceLeading` class that will contain and calculate voice leading information such as:

- a binary array showing changed voices
- sequence entropy of changes indicated by that array
- list of common notes
- list of changed notes
- numbers of common notes, changed notes, swaps
- change in span
- chord a and chord b
- numbers of up/down motions (to find voice leadings with more contrary motion)
- detect voice crossings
- max jump distance of changing voices

Also define a `get_voice_leading(chord_a, `list_of_chords`, n_shared_notes, min_bass_distance=0) -> List[VoiceLeading]` function. list_of_chords doesn't have to be from the same family as `chord_a`.

In [33]:
min_bass_distance = 2
n_shared_notes = 5

for ss in selection:
    for st in selection:
        for i in range(12):
            shifted = [(pc + i) % 12 for pc in st.pitch_classes]
            matches = [a for a, b in zip(ss.pitch_classes, shifted) if a == b]
            d1 = (shifted[0] - ss.pitch_classes[0]) % 12
            d2 = (ss.pitch_classes[0] - shifted[0]) % 12
            if (len(matches) == n_shared_notes) and (d1 > min_bass_distance) and (d2 > min_bass_distance):
                print(f"span: {ss.span}", ss.pitch_classes)
                print(f"span: {st.span}", shifted)
                print(f"common voices: {matches}")
                print(f"{len(matches)} common | {12 - len(matches)} changes | {n_swaps(shifted, ss.pitch_classes)} swaps")
                print(f"marked changes: {mark_changes(ss.pitch_classes, shifted)}")
                print(f"sequence entropy of changes: {calculate_sequence_entropy(mark_changes(ss.pitch_classes, shifted)):.2f}")
                print()

span: 56 [0, 4, 9, 3, 7, 1, 6, 11, 5, 10, 2, 8]
span: 59 [9, 3, 6, 0, 7, 1, 4, 10, 5, 11, 2, 8]
common voices: [7, 1, 5, 2, 8]
5 common | 7 changes | 2 swaps
marked changes: [1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0]
sequence entropy of changes: 24.11

span: 56 [0, 6, 10, 3, 9, 2, 7, 1, 5, 11, 4, 8]
span: 53 [3, 9, 0, 6, 10, 4, 7, 1, 5, 11, 2, 8]
common voices: [7, 1, 5, 11, 8]
5 common | 7 changes | 2 swaps
marked changes: [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0]
sequence entropy of changes: 22.78

span: 53 [0, 6, 9, 2, 7, 1, 4, 10, 3, 8, 11, 5]
span: 50 [3, 6, 11, 2, 8, 1, 7, 10, 4, 9, 0, 5]
common voices: [6, 2, 1, 10, 5]
5 common | 7 changes | 0 swaps
marked changes: [1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0]
sequence entropy of changes: 20.18

span: 50 [0, 5, 10, 4, 6, 11, 1, 7, 9, 3, 8, 2]
span: 43 [3, 8, 1, 4, 6, 11, 2, 7, 9, 0, 5, 10]
common voices: [4, 6, 11, 7, 9]
5 common | 7 changes | 4 swaps
marked changes: [1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1]
sequence entropy of changes: 24.11

span: 50 

## Generate Composition Material

### Epigraph

- Nothing changes because everything is already there (The history, empires, art, architecture, cradle of civilizations).
- Everything changes because chaos is the fountain from which cosmos emerges (Yet still creative, leading, etc.).

### Notes

Some properties of interest are:

scale:

- span
- sequence entropy
- number of intervals / allowed intervals

voice leading:

- sequence entropy of changes
- list of common notes
- list of tuples of changed notes
- numbers of common notes, changed notes, swaps
- change in span